## 6-3 6-4 6-5

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
#sl:satisfaction_level--False:MinMaxScaler,True:StandardScaler
#le:last_evaluation--False:MinMaxScaler,True:StandardScaler
#npr:number_project--False:MinMaxScaler,True:StandardScaler
#amh:average_montly_hours--
#tsc:time_spend_company--
#wa:Work_accident--
#pl5:promotion_last_5years--

#dp:department--False:LabelEncoding,True:OneHotEncoding
#slr:salary--
def hr_preprocessing(sl=False,le=False,npr=False,
                     amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,
                     lower_d=False,ld_n=1):
    df=pd.read_csv(r'C:/HR.csv')
    
    #2. 清洗数据
    df=df.dropna(subset=['satisfaction_level','last_evaluation'])
    df=df[df['satisfaction_level']<=1][df['salary']!='nme']
    df=df.rename(columns={'sales':'department'})
    #1. 标注
    label=df["left"]
    df=df.drop("left",axis=1)
    #3. 特征选择
    #4. 特征处理
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst=['satisfaction_level','last_evaluation','number_project',
                'average_montly_hours','time_spend_company','Work_accident','promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
            
    scaler_lst=[slr,dp]
    column_lst=['salary','department']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=="salary":
                df[column_lst[i]]=[map_salary(s) for s in df["salary"].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
    
    if lower_d:
        #return LinearDiscriminantAnalysis(n_components=ld_n)
        return PCA(n_components=ld_n).fit_transform(df.values),label
    return df,label
d=dict([("low",0),('medium',1),('high',2)])
def map_salary(s):
    return d.get(s,0)

In [2]:
def hr_modeling(features,label):
    from sklearn.model_selection import train_test_split
    f_v=features.values
    l_v=label.values
    X_tt,X_validation,Y_tt,Y_validation=train_test_split(f_v,l_v,test_size=0.2)
    X_train,X_test,Y_train,Y_test=train_test_split(X_tt,Y_tt,test_size=0.25)
    print(len(X_train),len(X_validation),len(X_test))
    
    #KNN
    from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
    knn_clf=KNeighborsClassifier(n_neighbors=3)
    knn_clf_n5=KNeighborsClassifier(n_neighbors=5)
    knn_clf.fit(X_train,Y_train)
    knn_clf_n5.fit(X_train,Y_train)
    Y_pred=knn_clf.predict(X_validation)
    Y_pred_n5=knn_clf_n5.predict(X_validation)
    from sklearn.metrics import accuracy_score,recall_score,f1_score
    print("验证集")
    print("ACC:",accuracy_score(Y_validation,Y_pred))
    print("REC:",recall_score(Y_validation,Y_pred))
    print("F-Score:",f1_score(Y_validation,Y_pred))
    print("--n=5---")
    print("ACC:",accuracy_score(Y_validation,Y_pred_n5))
    print("REC:",recall_score(Y_validation,Y_pred_n5))
    print("F-Score:",f1_score(Y_validation,Y_pred_n5))
    
    #用测试集衡量
    Y_pred_test=knn_clf.predict(X_test)
    print("测试得分")
    print("ACC:",accuracy_score(Y_test,Y_pred_test))
    print("REC:",recall_score(Y_test,Y_pred_test))
    print("F-Score:",f1_score(Y_test,Y_pred_test))
    
    #模型存储
    from sklearn.externals import joblib
    joblib.dump(knn_clf,"knn_clf") #保存
    #knn_clf=joblib.load("knn_clf") #使用时再次加载

In [3]:
def main():
    features,label=hr_preprocessing()
    hr_modeling(features,label)
if __name__=='__main__':
    main()

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


8999 3000 3000
验证集
ACC: 0.9523333333333334
REC: 0.9145658263305322
F-Score: 0.9013112491373361
--n=5---
ACC: 0.9463333333333334
REC: 0.8865546218487395
F-Score: 0.8871758934828311
测试得分
ACC: 0.952
REC: 0.9270687237026648
F-Score: 0.9017735334242838


由上面得分可知，n=3比n=5好一些

上面特征都是经过归一化处理的
下面看看另一种

In [4]:
def main():
    features,label=hr_preprocessing(sl=True,le=True,npr=True,
                     amh=True,tsc=True,wa=True,pl5=True,dp=True,slr=False
                     )
    hr_modeling(features,label)
if __name__=='__main__':
    main()

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


8999 3000 3000
验证集
ACC: 0.9573333333333334
REC: 0.927461139896373
F-Score: 0.9179487179487179
--n=5---
ACC: 0.9506666666666667
REC: 0.9080310880829016
F-Score: 0.904516129032258
测试得分
ACC: 0.949
REC: 0.9276595744680851
F-Score: 0.8952772073921972


In [5]:
#只对部门进行one-hot编码
def main():
    features,label=hr_preprocessing(dp=True)
    hr_modeling(features,label)
if __name__=='__main__':
    main()

8999 3000 3000


D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


验证集
ACC: 0.943
REC: 0.9232954545454546
F-Score: 0.8837525492861998
--n=5---
ACC: 0.9366666666666666
REC: 0.890625
F-Score: 0.868421052631579
测试得分
ACC: 0.943
REC: 0.9170182841068917
F-Score: 0.8840677966101694


模型存储

In [6]:
from sklearn.externals import joblib
#joblib.dump(knn_clf,"knn_clf") #保存
knn_clf=joblib.load("knn_clf") #使用时再次加载

6-4 分类-朴素贝叶斯

In [7]:
def hr_models(features,label):
    from sklearn.model_selection import train_test_split
    f_v=features.values
    l_v=label.values
    X_tt,X_validation,Y_tt,Y_validation=train_test_split(f_v,l_v,test_size=0.2)
    X_train,X_test,Y_train,Y_test=train_test_split(X_tt,Y_tt,test_size=0.25)
    print(len(X_train),len(X_validation),len(X_test))
    
    from sklearn.metrics import accuracy_score,recall_score,f1_score
    from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
    from sklearn.naive_bayes import GaussianNB,BernoulliNB #高斯朴素贝叶斯，伯努利朴素贝叶斯
    from sklearn.tree import DecisionTreeClassifier
    models=[]
    models.append(("KNN",KNeighborsClassifier(n_neighbors=3)))
    models.append(("GaussianNB",GaussianNB()))
    models.append(("BernoulliNB",BernoulliNB()))
    models.append(("DecisionTreeClassifier",DecisionTreeClassifier()))
    for clf_name,clf in models:
        clf.fit(X_train,Y_train)
        xy_lst=[(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]
        for i in range(len(xy_lst)):
            X_part=xy_lst[i][0]
            Y_part=xy_lst[i][1]
            Y_pred=clf.predict(X_part)
            print(i) #0:训练集，1：验证集，2：测试集
            print(clf_name,"-ACC:",accuracy_score(Y_part,Y_pred))
            print(clf_name,"-REC:",recall_score(Y_part,Y_pred))
            print(clf_name,"-F1:",f1_score(Y_part,Y_pred))

In [8]:
def main():
    features,label=hr_preprocessing(dp=True)
    hr_models(features,label)
if __name__=='__main__':
    main()

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


8999 3000 3000
0
KNN -ACC: 0.9705522835870652
KNN -REC: 0.9578947368421052
KNN -F1: 0.940462817344417
1
KNN -ACC: 0.947
KNN -REC: 0.9043348281016442
KNN -F1: 0.8838568298027758
2
KNN -ACC: 0.9443333333333334
KNN -REC: 0.9163179916317992
KNN -F1: 0.8872383524645509
0
GaussianNB -ACC: 0.7368596510723414
GaussianNB -REC: 0.7775743707093822
GaussianNB -F1: 0.5893166840097122
1
GaussianNB -ACC: 0.7253333333333334
GaussianNB -REC: 0.7832585949177877
GaussianNB -F1: 0.5598290598290598
2
GaussianNB -ACC: 0.729
GaussianNB -REC: 0.7670850767085077
GaussianNB -F1: 0.575013068478829
0
BernoulliNB -ACC: 0.8389821091232359
BernoulliNB -REC: 0.4759725400457666
BernoulliNB -F1: 0.5894020969113063
1
BernoulliNB -ACC: 0.8483333333333334
BernoulliNB -REC: 0.47533632286995514
BernoulliNB -F1: 0.5829514207149404
2
BernoulliNB -ACC: 0.8406666666666667
BernoulliNB -REC: 0.4839609483960948
BernoulliNB -F1: 0.5921501706484642
0
DecisionTreeClassifier -ACC: 1.0
DecisionTreeClassifier -REC: 1.0
DecisionTreeClass

## 分类决策树

http://graphviz.org/

In [9]:
import os
os.environ["PATH"]+=os.pathsep+"D:\Program Files\graphviz-2.38\release\bin"

* cmd下运行：conda install -c conda-forge pydotplus

In [10]:
import pydotplus

In [11]:
features,label=hr_preprocessing(dp=True)

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
from sklearn.model_selection import train_test_split
f_v=features.values
l_v=label.values
X_tt,X_validation,Y_tt,Y_validation=train_test_split(f_v,l_v,test_size=0.2)
X_train,X_test,Y_train,Y_test=train_test_split(X_tt,Y_tt,test_size=0.25)
print(len(X_train),len(X_validation),len(X_test))

8999 3000 3000


In [13]:
from sklearn.metrics import accuracy_score,recall_score,f1_score
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.externals.six import StringIO
dtree_clf=DecisionTreeClassifier()
dtree_clf.fit(X_train,Y_train)
Y_pred=dtree_clf.predict(X_validation)
print("----DecisionTreeGini----")
print("-ACC:",accuracy_score(Y_validation,Y_pred))
print("-REC:",recall_score(Y_validation,Y_pred))
print("-F1:",f1_score(Y_validation,Y_pred))

----DecisionTreeGini----
-ACC: 0.973
-REC: 0.9615384615384616
-F1: 0.9433962264150944


In [14]:
dot_data=export_graphviz(dtree_clf,out_file=None,
                        feature_names=features.columns.values,
                        class_names=["NL","L"],
                        filled=True,
                        rounded=True,
                        special_characters=True
                )

In [15]:
graph=pydotplus.graph_from_dot_data(dot_data)

In [16]:
graph.write_pdf("dt_tree.pdf")

True

画决策树的另外一种写法

In [17]:
dot_data=StringIO()
export_graphviz(dtree_clf,out_file=dot_data,
                        feature_names=features.columns.values,
                        class_names=["NL","L"],
                        filled=True,
                        rounded=True,
                        special_characters=True
                )
graph=pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("dt_tree_2.pdf")

True

DecisionTreeEntropy

In [18]:
dtree_clf=DecisionTreeClassifier(criterion='entropy')
dtree_clf.fit(X_train,Y_train)
Y_pred=dtree_clf.predict(X_validation)
print("----DecisionTreeClassifier----")
print("-ACC:",accuracy_score(Y_validation,Y_pred))
print("-REC:",recall_score(Y_validation,Y_pred))
print("-F1:",f1_score(Y_validation,Y_pred))

----DecisionTreeClassifier----
-ACC: 0.9756666666666667
-REC: 0.9615384615384616
-F1: 0.9486999297259311


调参

In [19]:
dtree_clf=DecisionTreeClassifier(criterion='gini',min_impurity_split=0.1)
dtree_clf.fit(X_train,Y_train)
Y_pred=dtree_clf.predict(X_validation)
print("----DecisionTreeClassifier----")
print("-ACC:",accuracy_score(Y_validation,Y_pred))
print("-REC:",recall_score(Y_validation,Y_pred))
print("-F1:",f1_score(Y_validation,Y_pred))

----DecisionTreeClassifier----
-ACC: 0.975
-REC: 0.9316239316239316
-F1: 0.9457700650759219


D:\Anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


## 6-6 分类-支持向量机

In [20]:
from sklearn.svm import SVC
svc_clf=SVC(C=1000)
svc_clf.fit(X_train,Y_train)
Y_pred = svc_clf.predict(X_validation)
print("----SVM--Classifier----")
print("-ACC:",accuracy_score(Y_validation,Y_pred))
print("-REC:",recall_score(Y_validation,Y_pred))
print("-F1:",f1_score(Y_validation,Y_pred))

----SVM--Classifier----
-ACC: 0.9616666666666667
-REC: 0.9188034188034188
-F1: 0.9181494661921707


https://www.bilibili.com/video/av38197896/?p=67

## 6-7 分类-集成-随机森林

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf_clf=RandomForestClassifier(n_estimators=10,max_features="auto",bootstrap=True)
rf_clf.fit(X_train,Y_train)
Y_pred=rf_clf.predict(X_validation)
print("---RandomForestClassifier---")
print("ACC",accuracy_score(Y_validation,Y_pred))
print("REC",recall_score(Y_validation,Y_pred))
print("F1",f1_score(Y_validation,Y_pred))

---RandomForestClassifier---
ACC 0.9846666666666667
REC 0.9501424501424501
F1 0.9666666666666667


## 6-8 分类-集成-Adaboost

提升法（boost)
Adaboost优点：
* 精度高，且灵活可调
* 几乎不用担心过拟合
* 简化特征工程流程

In [35]:
from sklearn.ensemble import AdaBoostClassifier
adb_clf=AdaBoostClassifier(n_estimators=100)
adb_clf.fit(X_train,Y_train)
Y_pred=adb_clf.predict(X_validation)
print("---AdaBoostClassifier---")
print("ACC",accuracy_score(Y_validation,Y_pred))
print("REC",recall_score(Y_validation,Y_pred))
print("F1",f1_score(Y_validation,Y_pred))

---AdaBoostClassifier---
ACC 0.962
REC 0.9216524216524217
F1 0.9190340909090909


测试集

In [38]:
adaboost_clf=AdaBoostClassifier(base_estimator=SVC(),algorithm="SAMME")
adaboost_clf.fit(X_train,Y_train)
Y_pred_test=adaboost_clf.predict(X_test)
print("acc",accuracy_score(Y_test,Y_pred_test))
print("rec",recall_score(Y_test,Y_pred_test))
print("F1",f1_score(Y_test,Y_pred_test))

acc 0.7696666666666667
rec 0.0
F1 0.0


D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## 6-9 回归-线性回归